# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [62]:
import requests

url = "https://www.rottentomatoes.com/"
response = requests.get(url)
print(response.status_code)
print(response.headers['Content-Type'])

from bs4 import BeautifulSoup

soup = BeautifulSoup(response.content, "html.parser")

print(soup.find('a', attrs={'slot':'caption'}))
# soup.find_all("tile-dynamic",attrs={'data-qa':"tile"})[0]
#print(soup.prettify())

200
text/html; charset=utf-8
<a data-track="scores" href="/m/kingdom_of_the_planet_of_the_apes" slot="caption">
<score-pairs-deprecated audiencescore="hide" audiencesentiment="hide" criticscertified="" criticsscore="81" criticssentiment="positive">
</score-pairs-deprecated>
<span class="p--small">Kingdom of the Planet of the Apes</span>
</a>


In [59]:
new_soup = BeautifulSoup(requests.get(url+'tv/dark_matter_2024/s01').content, "html.parser")
print(new_soup.find('rt-text', attrs={'slot': "content"}).text)


                Jason Dessen is abducted into an alternate version of his life; to get back to his true family, he embarks on a harrowing journey to save them from the most terrifying foe imaginable: himself; based on Blake Crouch's best-selling book.
            


In [77]:
# Your solution goes here
import pandas as pd
def pass_filter(filter, link, link_type, media_type):
  if not filter: return True
  return filter and link_type.get(link.get('href').split('/')[1],'') == media_type

def scrape_rotten_tomatoes(soup,media_type='all',rating = 0.0):
  card_attrs = {'class':'transparent unset'}
  media_type = media_type.lower()
  link_type = {'m':'movie', 'tv':'tvSeason'} # for filtering out hrefs and ratings later
  filter = False
  if media_type != 'all':
    assert media_type == 'movie' or 'tv' in media_type,"not valid content type"+ f' {media_type}'
    filter = True
    if 'tv' in media_type.lower(): media_type = 'tvSeason' # tv shows are labeled tvSeason
    card_attrs['data-content-type'] = media_type

  # data = [data for data in soup.find_all("tile-dynamic",attrs={'data-qa':"tile"})]
  t_search = soup.find_all("button",attrs=card_attrs)
  titles = [title.span.text for title in t_search]

  hrefs = soup.find_all('a', attrs={'slot':'caption'})
  link_data = [(link.get('href'),i) for i,link in enumerate(hrefs) if pass_filter(filter,link,link_type,media_type) ]
  links = [data[0] for data in link_data]

  rows = {data[1] for data in link_data}
  raw_ratings = [rate.select('score-pairs-deprecated')[0].get('criticsscore') for i,rate in enumerate(hrefs) if i in rows]
  ratings = [float(rating) if rating else 0.0 for rating in raw_ratings]

  desc = []
  for link in links:
    title_request = requests.get(url+link[1:])
    if title_request.status_code != 200: desc.append(float('NaN')); continue
    desc_soup = BeautifulSoup(title_request.content, "html.parser")
    desc.append(desc_soup.find('rt-text', attrs={'slot': "content"}).text.strip())
  print(desc[0], len(desc))
  df = pd.DataFrame({'title': tuple(titles),
                     'ratings': tuple(ratings),
                     'link': tuple(links),
                     "Description": tuple(desc)})
  return df[df['ratings'] >= rating]

# great_tv = scrape_rotten_tomatoes(soup,media_type='tv',rating=90.0)
# great_movie = scrape_rotten_tomatoes(soup,media_type='movie',rating=90.0)

Jason Dessen is abducted into an alternate version of his life; to get back to his true family, he embarks on a harrowing journey to save them from the most terrifying foe imaginable: himself; based on Blake Crouch's best-selling book. 19


,title,ratings,link,Description
1,Doctor Who,96.0,/tv/doctor_who_2023/s01,The Doctor and friends travel from the dawn of...
2,Hacks,97.0,/tv/hacks/s03,"A year after parting, Deborah Vance (Jean Smar..."
3,Star Wars: Tales of the Empire,85.0,/tv/star_wars_tales_of_the_empire/s01,"After losing everything, young Morgan Elsbeth ..."
4,Them,100.0,/tv/them/s02,A Black family moves from North Carolina to an...
5,Welcome to Wrexham,100.0,/tv/welcome_to_wrexham/s03,"In ""Welcome to Wrexham,"" Rob McElhenney and Ry..."
9,Baby Reindeer,98.0,/tv/baby_reindeer/s01,"Dealing with a female stalker, a man is forced..."
10,Dead Boy Detectives,93.0,/tv/dead_boy_detectives/s01,Two teenage ghosts work alongside a clairvoyan...
11,The Big Door Prize,93.0,/tv/the_big_door_prize/s02,A small town is forever changed when a mysteri...
12,X-Men '97,98.0,/tv/x_men_97/s01,A band of mutants use their uncanny gifts to p...
15,Fallout,93.0,/tv/fallout/s01,Fallout is the story of haves and have-nots in...


## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [ ]:
# Your solution goes here